# 00_plan Prompt API Playground

Scope: API-only prompt testing for `harness/prompts/00_plan`.

This notebook does **not** create projects, does **not** call the orchestrator, and does **not** invoke pipeline phases.

## Cell 1: Setup (Out-of-the-box)

Purpose:
- Make imports work even when notebook is run from `/notebooks`.
- Load `.env` from repo root automatically.
- Import the same harness prompt composer and API client used in production.

In [1]:
import os
import sys
from pathlib import Path

# Resolve repo root whether cwd is repo root, notebooks/, or another child folder.
cwd = Path.cwd().resolve()
candidates = [cwd, *cwd.parents]
repo_root = None
for p in candidates:
    if (p / 'harness').exists() and (p / 'notebooks').exists():
        repo_root = p
        break
if repo_root is None:
    raise RuntimeError('Could not locate repo root containing harness/ and notebooks/')
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

# Load environment from repo .env if possible.
env_path = repo_root / '.env'
try:
    from dotenv import load_dotenv
    if env_path.exists():
        load_dotenv(env_path)
except Exception:
    pass

from harness.prompts import compose_plan_prompt
from harness.client import LLMClient, estimate_tokens

print(f'Repo root: {repo_root}')

Repo root: /Users/velocityworks/IdeaProjects/flaming-horse


## Cell 2: Editable Request Inputs

Purpose:
- Set topic and generation knobs for the API request.
- Defaults are configured to perform a real API call when you run later cells.

In [2]:
# Topic value fills {{topic}} in 00_plan/user.md.
topic = "How matrix multiplication works"

# API generation parameters.
temperature = 0.7
max_tokens = 16000


## Cell 3: Compose Prompts (Exact Harness Logic)

Purpose:
- Build `system_prompt` + `user_prompt` exactly as harness `plan` phase does.
- Show size estimates before spending tokens.

In [3]:
# Minimal state placeholder; compose_plan_prompt currently doesn't require state fields.
state = {}

system_prompt, user_prompt = compose_plan_prompt(state=state, topic=topic)

print(f'System prompt chars: {len(system_prompt):,}')
print(f'User prompt chars:   {len(user_prompt):,}')
print(f'System est tokens:   {estimate_tokens(system_prompt):,}')
print(f'User est tokens:     {estimate_tokens(user_prompt):,}')

System prompt chars: 400
User prompt chars:   949
System est tokens:   100
User est tokens:     237


## Cell 4: Prompt Preview

Purpose:
- Quickly inspect rendered prompts before sending to API.

In [4]:
print('=== SYSTEM PROMPT PREVIEW ===')
print(system_prompt[:2000])
print('\n=== USER PROMPT PREVIEW ===')
print(user_prompt[:2000])

=== SYSTEM PROMPT PREVIEW ===
You are an expert educational video planner.

Your job is to produce clear, teachable scene plans that progress logically from fundamentals to deeper ideas.

Behavior:
- Focus on conceptual clarity and audience comprehension.
- Prefer concrete, topic-relevant visuals over abstract filler.
- Keep scene-to-scene flow coherent and cumulative.
- Ensure each scene has a specific instructional purpose.


=== USER PROMPT PREVIEW ===
Create a video plan for this topic:
How matrix multiplication works

Return one JSON object with:
- title (string)
- description (string)
- target_duration_seconds (integer)
- scenes (array)

Each scene must include:
- id (string, pattern: scene_XX_slug)
- title (string)
- narration_key (string; usually same as id)
- description (string)
- estimated_duration_seconds (integer)
- visual_ideas (array of strings)

Planning constraints:
- 8-12 scenes
- 20-45 seconds per scene
- total target duration 240-480 seconds
- no empty fields
- visu

## Cell 5: API Call

Purpose:
- Perform a real LLM call using harness client config from env.
- Return raw response text only (no parsing/writing artifacts).

In [5]:
provider = os.getenv('LLM_PROVIDER', 'XAI').upper()
api_key_var = f'{provider}_API_KEY'
assert os.getenv(api_key_var), f'Missing {api_key_var} in environment/.env'

client = LLMClient()
response_text = client.chat_completion(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    temperature=temperature,
    max_tokens=max_tokens,
)

print(response_text[:6000])

🤖 Harness using:
   Provider: XAI
   Base URL: https://api.x.ai/v1
   Model: grok-4-1-fast-reasoning
{"title":"How Matrix Multiplication Works","description":"A step-by-step guide to understanding matrix multiplication, starting from matrix basics and building to full examples with clear visualizations.","target_duration_seconds":360,"scenes":[{"id":"scene_01_intro","title":"Introduction to Matrix Multiplication","narration_key":"scene_01_intro","description":"Introduce the topic, explain why matrix multiplication is important in math, computing, and graphics, and preview the video structure.","estimated_duration_seconds":25,"visual_ideas":["Title text fades in with matrix grid icons","Two sample matrices A (2x2) and B (2x2) slide in from sides","Quick animation of C = A * B result matrix appearing"]},{"id":"scene_02_matrix_basics","title":"What is a Matrix?","narration_key":"scene_02_matrix_basics","description":"Define a matrix as a rectangular array of numbers, show notation like A 